In [ ]:
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM "file:/Users/markneedham/projects/neo4j-northwind/data/customerDb.csv" AS row
MERGE (employee:Employee {employeeId: row.EmployeeID})
MERGE (order:Order {orderId: row.OrderID})
MERGE (employee)-[:SOLD]->(order)

In [ ]:
LOAD CSV FROM "file:///data/asoto/projectW205/data/MillionSongSubset/graph/nodes_artists.csv" AS line
WITH line LIMIT 1 RETURN 1

In [ ]:
// This is a comment in cypher
CREATE CONSTRAINT ON (a:ARTIST) ASSERT a.id IS UNIQUE;
CREATE CONSTRAINT ON (s:SONG) ASSERT s.id IS UNIQUE;
CREATE CONSTRAINT ON (y:YEAR) ASSERT y.year IS UNIQUE;
CREATE CONSTRAINT ON (t:TAG) ASSERT t.tag IS UNIQUE;
CREATE CONSTRAINT ON (a:ALBUM) ASSERT a.name IS UNIQUE;

# Start Here

In [1]:
from py2neo import Graph
from py2neo import authenticate
import os

In [2]:
os.environ['INSTANCE_PDNS']

'ec2-54-90-132-249.compute-1.amazonaws.com'

In [4]:
os.environ['HOME']

'/home/asoto'

In [4]:
"http://"+pdns+":7474/db/data/"

'http://ec2-54-90-132-249.compute-1.amazonaws.com:7474/db/data/'

In [5]:
os.getcwd()

'/data/asoto/projectW205'

In [3]:
# set up authentication parameters
pdns = os.environ['INSTANCE_PDNS']
#authenticate("ec2-54-90-132-249.compute-1.amazonaws.com:7474", "neo4j", "neo")

# connect to authenticated graph database
graph = Graph("http://"+pdns+":7474/db/data/")

In [11]:
unique_constraints = [('ARTIST', 'id'),('SONG','id'),('ALBUM','name'),('TAG','tag'),('YEAR','year')]

for label,prop in unique_constraints:
    try:
        graph.schema.create_uniqueness_constraint(label, prop)
    except:
        pass

In [37]:
# Artists
csv_path = 'file:' + os.getcwd() + '/data/MillionSongSubset/graph/nodes_artists.csv'

tx = graph.cypher.begin()

st = "USING PERIODIC COMMIT 1000 "
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += "MERGE (a:ARTIST {id:line.id}) "
st += "ON CREATE SET a.idmb=line.mbid, a.id7d=line.id7d, a.name=lower(line.name);"

tx.append(st)
tx.commit()

In [38]:
# Songs
csv_path = 'file:' + os.getcwd() + '/data/MillionSongSubset/graph/nodes_songs.csv'

tx = graph.cypher.begin()

st = "USING PERIODIC COMMIT 1000 "
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += "MERGE (s:SONG {id:line.id}) "
st += "ON CREATE SET s.tackid=line.trackid, s.title=lower(line.name), s.danceability=TOFLOAT(line.dance),"
st += "s.duration = TOFLOAT(line.dur), s.energy= TOFLOAT(line.energy), s.loudness = TOFLOAT(line.loudness);"

tx.append(st)
tx.commit()

In [39]:
# Albums
csv_path = 'file:' + os.getcwd() + '/data/MillionSongSubset/graph/nodes_albums.csv'

tx = graph.cypher.begin()

st = "USING PERIODIC COMMIT 1000 "
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += "MERGE (a:ALBUM {name: lower(line.name)});"

tx.append(st)
tx.commit()

In [49]:
# Tags
csv_path = 'file:' + os.getcwd() + '/data/MillionSongSubset/graph/nodes_tags.csv'

tx = graph.cypher.begin()

st = "USING PERIODIC COMMIT 1000 "
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += "MERGE (:TAG {tag: lower(line.tag)});"

tx.append(st)
tx.commit()

In [45]:
# Year
csv_path = 'file:' + os.getcwd() + '/data/MillionSongSubset/graph/nodes_years.csv'

tx = graph.cypher.begin()

st = "USING PERIODIC COMMIT 1000 "
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += "MERGE (y:YEAR {year: TOINT(line.year)});"

tx.append(st)
tx.commit()

### Relationships

In [51]:
# (ARTIST)-[SIMILAR_TO]-> (ARTIST)
csv_path = 'file:' + os.getcwd() + '/data/MillionSongSubset/graph/rel_similar_artists.csv'

tx = graph.cypher.begin()

st = "USING PERIODIC COMMIT 1000 "
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += "MATCH (from:ARTIST {id:line.from}) "
st += 'MATCH (to:ARTIST {id:line.to})'
st += 'MERGE (from)-[:SIMILAR_TO]->(to)'

tx.append(st)
tx.commit()

In [52]:
# (ARTIST)-[PERFORMS]-> (SONG)
csv_path = 'file:' + os.getcwd() + '/data/MillionSongSubset/graph/rel_performs.csv'

tx = graph.cypher.begin()

st = "USING PERIODIC COMMIT 1000 "
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += "MATCH (a:ARTIST {id:line.artist}) "
st += 'MATCH (s:SONG {id:line.song})'
st += 'MERGE (a)-[:PERFORMS]->(s)'

tx.append(st)
tx.commit()

In [12]:
# (ARTIST)-[HAS_ALBUM]-> (ALBUM)
csv_path = 'file:' + os.getcwd() + '/data/MillionSongSubset/graph/rel_artist_has_album.csv'

tx = graph.cypher.begin()

st = "USING PERIODIC COMMIT 1000 "
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += "MATCH (a:ARTIST {id:line.artistId}) "
st += 'MATCH (m:ALBUM {name:lower(line.album)})'
st += 'MERGE (a)-[:HAS_ALBUM]->(m);'

tx.append(st)
tx.commit()

In [13]:
# (SONG)-[IN_ALBUM]-> (ALBUM)
csv_path = 'file:' + os.getcwd() + '/data/MillionSongSubset/graph/rel_song_in_album.csv'

tx = graph.cypher.begin()

st = "USING PERIODIC COMMIT 1000 "
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MATCH (a:ALBUM {name:lower(line.album)}) '
st += 'MATCH (s:SONG {id:line.songID}) '
st += 'MERGE (s)-[:IN_ALBUM]->(a);'

tx.append(st)
tx.commit()

In [ ]:
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 
"file:/data/asoto/projectW205/data/MillionSongSubset/graph/rel_similar_artists.csv" AS line
MATCH (from:ARTIST {id:line.from})
MATCH (to:ARTIST {id:line.to})
MERGE (from)-[:SIMILAR_TO]->(to)

In [50]:
'file:' + os.getcwd() + '/data/MillionSongSubset/graph/rel_similar_artists.csv '

'file:/data/asoto/projectW205/data/MillionSongSubset/graph/rel_similar_artists.csv '